In [1]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_n


문제 2-1 : 콤마 구분 리스트 파서 활용

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="{subject}와(과) 관련된 한국의 유명한 장소나 활동 5가지를 추천해줘.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

chain = prompt | llm | output_parser

user_input = "음식"
# input("관심 분야를 입력하세요 (예: 음식, 스포츠, 영화): ")

result = chain.invoke({"subject": user_input})

print("예시 입력: " + user_input)
print("예시 출력:")
print(result)

예시 입력: 음식
예시 출력:
['서울의 한옥마을 음식거리', '부산의 광복동 먹자골목', '전주한정식마을', '여수 먹거리 골목', '경주의 황남빵마을']


문제 2-2 : 영화 리뷰 감정 분석기

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum

class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

template = """
당신은 영화 리뷰 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

In [6]:

def safe_sentiment_analysis(text, use_fixing_parser=True):
    try:
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

def run_sentiment_analysis():
    print("테스트 리뷰 예시:")

    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. {text}")
        
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

테스트 리뷰 예시:

1. 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 

=== 결과 요약 ===
성공: 3/3 (100.0%)
실패: 0/3


문제 2-3 : 학생 정보 구조화 시스템

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

class SelfIntroduction(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="목표")

parser = PydanticOutputParser(pydantic_object=SelfIntroduction)

template = """
다음 사용자 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태의 자기소개서를 완성해 주세요.
입력: {input}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

input = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"input": input})

pprint(output.model_dump())

{'age': 22,
 'goal': '훌륭한 개발자가 되는 것',
 'hobbies': ['게임하기', '영화보기', '코딩'],
 'major': '컴퓨터공학',
 'name': '김민수'}


문제 2-4 : 여행 계획 분석기

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

response_schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="여행 예산"),
    ResponseSchema(name="rating", description="1-5점 사이의 추천도"),
    ResponseSchema(name="activities", description="주요 활동 리스트", type="list[str]")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

template = """
다음 여행 후기나 계획 텍스트를 분석하세요. 여행 후기나 계획 텍스트: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

review = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

chain = prompt | model | parser

output = chain.invoke({"review": review})
pprint(output)

{'activities': ['해운대 바다 구경', '자갈치시장에서 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4'}
